## NLP with DL

## Import libraries

## Train Test Split

## Tokenization

## Creating word index

## Converting tokens to numeric

## Maximum number of tokens for all documents

## Fixing token counts of all documents (pad_sequences)

## Converting numeric tokens to string

## Modelling

## Model evaluation

## Prediction